# RSS Wikipedia Search Dashboard

This notebook is a final product of work on two other notebooks:

1. Dev - 1 - Fasttext wikipedia search
2. Dev - 2 - Combining search types for text and wiki

These 2 notebooks are intermediate development notebooks, final production content is in this notebook.

Here, you will find a detailed description of the 4 search algorithms we use to find relevant Wikipedia articles for the newsfeed titles.

The high-level contents of the notebook are following: 

1. Initialize Danish Wikipedia
2. Initialize RsspediaInit class
3. Initialize the RSSWikiDashboard class

Under each heading, you will find a detailed description of the processes and algorithms involved.

**Global setup**

In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [ ]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height: 10000px;
}
</style>

**Import modules**

In [ ]:
from src.text.document_retrieval.wikipedia import Wikipedia
from notebooks.exercises.src.text.news_wiki_search_init import RsspediaInit
from notebooks.exercises.src.text.news_wiki_dashboard import RSSWikiDashboard

## 1. Initialize Danish Wikipedia

In [ ]:
wikipedia = Wikipedia(
    language="Danish",
    cache_directory_url=False
)

## 2. Initialize RsspediaInit class

In this initialization, the data that is needed to perform search is loaded.
1. Okapi BM-25: uses precomputed wikipedia tf and idf vectors. No more preprocessing is done.
2. Explicit Semantic Analysis: here we load the stored tf-idf vectors or compute them and store on disk.
3. FTN-a and FTN-b: here we load the stored Fasttext vectors for wikipedia titles and abstracts or compute them and store on disk. As a preprocessing, non-alphanumeric characters are removed, and zero-length abstracts are removed as well. Wikipedia documents and titles are adjusted accordingly.
<br><br>
For both (2) and (3) we remove the following stop-words: <br>
<code>stop_words = ["den", "det", "denne", "dette", "en", "et", "om", "for", "til", "at", "af", "på", "som", "og", "er", "i"]</code>

In [ ]:
rss_search_init = RsspediaInit(wikipedia = wikipedia)

## 3. Initialize and start the RSSWikiDashboard

### 3.1 Types of search and their descriptions

Before searching the wikipedia, both wikipedia page texts and searchable texts are preprocessed:
* Remove stopwords
* Remove non-alphanumeric characters

<code>self.texts_clean = [self.getCleanWordsList(self.texts[i], return_string = True) for i in range(len(self.texts))]</code><br>

#### 3.1.1 Okapi BM-25

In information retrieval, Okapi BM25 (BM stands for Best Matching) is a ranking function used by search engines to rank matching documents according to their relevance to a given search query. It is based on the probabilistic retrieval framework developed in the 1970s and 1980s by Stephen E. Robertson, Karen Spärck Jones, and others.

The name of the actual ranking function is BM25. To set the right context, however, it is usually referred to as "Okapi BM25", since the Okapi information retrieval system, implemented at London's City University in the 1980s and 1990s, was the first system to implement this function.

BM25 and its newer variants, e.g. BM25F (a version of BM25 that can take document structure and anchor text into account), represent state-of-the-art TF-IDF-like retrieval functions used in document retrieval.

Given a query $Q$, containing keywords $ q_1,...,q_n$, the BM25 score of a document $D$ is:

$$
score(D,Q)=\sum_{i=1}^n IDF(q_i) * \frac{f(q_i,D)*(k_1+1)} {f(q_i,D)+k_1*(1-b+b*\frac{|D|}{(avgdl)}},
$$

where $f(q_i,D)$ is $q_{i}$'s term frequency in the document $D$, $|D|$ is the length of the document $D$ in words, and $avgdl$ is the average document length in the text collection from which documents are drawn. $k_1$ and $b$ are free parameters, usually chosen, in absence of an advanced optimization, as $k_1\in [1.2,2.0]$ and $b=0.75$. $IDF(q_i)$ is the IDF (inverse document frequency) weight of the query term $q_i$. 

It is usually computed as:

$$
IDF(q_i)=log((N-n(q_i)+0.5)/(n(q_i)+0.5),
$$

where $N$ is the total number of documents in the collection, and $n(q_i)$ is the number of documents containing $q_i$.


#### 3.1.2 Explicit Semantic Analysis

**First, we preprocess the texts.**
* Remove line breaks (symbols \r\n) and replace them with whitespaces

<code>pattern = re.compile('[\n\r ]+', re.UNICODE)</code><br>
<code>self.texts = [pattern.sub(' ', self.texts[i]) for i in range(len(self.texts))]</code><br>

**Second, we compute the TF-IDF representation of Danish Wikipedia article full texts.**
* Initialize the TfidfVectorizer with L2 normalization
* Transform the preprocessed texts into TF-IDF representations

<code>self._transformer = TfidfVectorizer(stop_words = None, norm = "l2", use_idf = True, sublinear_tf = False)</code><br>
<code>self._Y = self._transformer.fit_transform(self.texts_clean)</code>

**Third, we compute the TF-IDF representation of the searchable text.**

<code>y = self._transformer.transform([text])</code><br>

**Fourth, we multiply the two sparce matrices and convert the result to a dense matrix (as np.array).**

<code>D = np.array((self._Y * y.T).todense())</code>

**Finally, we sort the results and get the indices of our matches.**

<code>indices = np.argsort(-D, axis=0)</code>


#### 3.1.3 FTN-a

**First, we compute the vectorized representation of the Wikipedia title texts and the searchable text by summing up the representations of individual words**

<code>text_vector = text_vector + self.model.wv[words[i]]</code><br>

**Second, we compute the cosine distance between the searchable text vector and all the Wikipedia title vectors.**

<code>cdist_list = cdist(wiki_title_vectors, searchable_text_vector, 'cosine')</code><br>

**Third, we sort the list of distances and get the results**

<code>cdist_list_sorted = np.sort(cdist_list, axis = 0)</code><br>
<code>result = np.where(cdist_list == cdist_list_sorted[i])</code><br>


#### 3.1.4 FTN-b


**First, we compute the vectorized representation of the Wikipedia title texts.**

<code>text_vector = text_vector + self.model.wv[words[i]]</code><br>

**Second, we extract the n-grams (1 and 2 words) from the searchable text.**

<code>ngrams = self.get_ngrams(text)</code><br>

**For each n-gram, we compute the vectorized representation of that n-gram by summing up the vectors of individual words.**

<code>text_vector = text_vector + self.model.wv[words[i]]</code><br>

**Then, we compute the cosine distance between the searchable n-gram vector and all the Wikipedia title vectors and between the n-gram vector and the searchable text and combine the two using a parameter $p$.**

This is done in order for n-grams to have lower cosine distance to the Wikipedia title, because the n-gram will have a lower cosine distance with the whole searchable text than a part of the n-gram.

<code>cdist_list1 =  cdist(wiki_title_vectors, n_gram_vector, 'cosine')</code><br>
<code>cdist_list2 =  cdist(searchable_text_vector, n_gram_vector, 'cosine')</code><br>
<code>cdist_list = (cdist_list1 * p + cdist_list2 * (1 - p))</code><br>

**Last, we sort the list of distances and get the results**

<code>cdist_list_sorted = np.sort(cdist_list, axis = 0)</code><br>
<code>result = np.where(cdist_list == cdist_list_sorted[i])</code><br>


### 3.2 Post-processing

Here, there is an option to exclude too similar results. Some wikipedia pages are similar and refer to similar entities. 
Cosine distance is measured between all the pairs of results, and if the result similarity is higher than some threshold, then the result is removed.

In [ ]:
rsswdb = RSSWikiDashboard(wikipedia, rss_search_init)
rsswdb.start

In [ ]:
#r = rsswdb.rsspedia_search.cdist_func([rss_search_init.sumVectorRepresentation("6")],
#    [rss_search_init.sumVectorRepresentation("Guide Anmelderne anbefaler 6 gode spisesteder med pasta på menuen")])
#import pprint
#pprint.pprint(titles)